In [18]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import glob
import os
import subprocess
import datetime

In [26]:
all_data=glob.glob('/Users/nikkibytes/Documents/testing/sub*')
all_data
datestamp=datetime.datetime.now().strftime("%Y-%m-%d-%H_%M_%S")

writedir='/Users/nikkibytes/Documents/testing/sub-001/fmriprep/sub-001/func/motion_assessment'
 
outhtml = os.path.join(writedir,'bold_motion_QA_test_%s.html'%(datestamp))
out_bad_bold_list = os.path.join(writedir,'testing_%s.txt'%(datestamp))
for sub in all_data:
    print("SUBJECT: ", sub)
    for dir in glob.glob(os.path.join(sub,'fmriprep/sub-001/func')):#path to the functional, skull stripped data
        print("DIRECTORY: ", dir)#not needed but i get crazy 
        os.chdir(dir)
        if not os.path.exists(os.path.join(dir,'motion_assessment')): #looking for a motion assessment dir to put out put in, I like to put it in my functional directory where my skull stripped brain is
            os.makedirs(os.path.join(dir,'motion_assessment')) #making dir if it doesn't exist
        for brain_input in glob.glob(os.path.join('*brainmask.nii.gz')):
            id=brain_input.split('.')[0]
            output=os.path.join(dir, 'motion_assesment/'+id+'_confound.txt ')
            input_dir=os.path.join(dir, input)

            
            cmd="fslnvols " + brain_input
            comparator = subprocess.check_output(cmd, shell=True, encoding="utf-8")
            comparator = comparator.strip()
            comparator = int(comparator) *.25
            
            
            print("Identifier: ", id)
            print("Brainmask File(Input): ", brain_input)
            print("Comparison: ", comparator)
            print("CMD:", cmd)
            print("INPUT DIR: ", input_dir)
            print("OUTPUT: ", output)
                
                
            
            
            
            # this is generating the fd confounds txt, it is using the fd metric,
            # making a plot and putting it in the motion assessment directory we made above

            os.system("fsl_motion_outliers -i motion_assesment/%s_confound.txt -o %s --nomoco  --fd --thresh=0.9 -p motion_assessment/fd_plot -v > %s_outlier_output.txt"%(brain_input,id,id))
            
            

                
                
                ####this is writing the motion assessment fd metric to the html file
            os.system("cat motion_assessment/%s_outlier_output.txt >> %s"%(id,outhtml))
                
                ###getting the full path to the plot
            plotz=os.path.join(dir,'motion_assessment','fd_plot.png')
                
                
                #### putting the full plot in the html file
            os.system("echo '<p>=============<p>FD plot %s <br><IMG BORDER=0 SRC=%s WIDTH=%s></BODY></HTML>' >> %s"%(id,plotz,'100%', outhtml))
                
                
                ####sometimes you have a great subject who didn't move, in this case we want to make a blank file
            if os.path.isfile("motion_assessment/%s_confound.txt"%(id))==False:
                os.system("touch motion_assessment/%s_confound.txt"%(id))
            
            check = subprocess.check_output("grep -o 1 motion_assessment/%s_confound.txt | wc -l"%(id), shell=True) #how many columns are there = how many 'bad' points
            num_scrub = [int(s) for s in check.split() if s.isdigit()]

             
            if num_scrub[0] > comparator: #if the number in check is greater than num_scrub then we don't want it
                with open(out_bad_bold_list, "a") as myfile: #making a file that lists all the bad ones
                    myfile.write("%s\n"%(id))
                    print("wrote bad file")
                myfile.close()


SUBJECT:  /Users/nikkibytes/Documents/testing/sub-001
DIRECTORY:  /Users/nikkibytes/Documents/testing/sub-001/fmriprep/sub-001/func
Identifier:  sub-001_task-rest_bold_space-T1w_brainmask
Brainmask File(Input):  sub-001_task-rest_bold_space-T1w_brainmask.nii.gz
Comparison:  0.25
CMD: fslnvols sub-001_task-rest_bold_space-T1w_brainmask.nii.gz
INPUT DIR:  /Users/nikkibytes/Documents/testing/sub-001/fmriprep/sub-001/func/sub-001_task-prob_run-2_bold_space-T1w_brainmask.nii.gz
OUTPUT:  /Users/nikkibytes/Documents/testing/sub-001/fmriprep/sub-001/func/motion_assesment/sub-001_task-rest_bold_space-T1w_brainmask_confound.txt 
Identifier:  sub-001_task-prob_run-3_bold_space-T1w_brainmask
Brainmask File(Input):  sub-001_task-prob_run-3_bold_space-T1w_brainmask.nii.gz
Comparison:  0.25
CMD: fslnvols sub-001_task-prob_run-3_bold_space-T1w_brainmask.nii.gz
INPUT DIR:  /Users/nikkibytes/Documents/testing/sub-001/fmriprep/sub-001/func/sub-001_task-prob_run-2_bold_space-T1w_brainmask.nii.gz
OUTPUT:  

In [27]:
def main():
    sub_dir = input("Directory of Subjects: ")
    print(sub_dir)

In [30]:
raw_input("enter name: ")

NameError: name 'raw_input' is not defined